In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
import fiona

In [2]:
%%time
df = pd.read_csv('data/df_copy_3.csv',usecols=['RowID','UA','Latitude','Longitude']) 
len(df)

CPU times: user 22.4 s, sys: 1.27 s, total: 23.7 s
Wall time: 23 s


14733437

In [3]:
df2 = df.tail(1000)

In [4]:
%%time
geometry = [Point(xy) for xy in zip(df2['Longitude'], df2['Latitude'])]
crs = {'init': 'epsg:2263'} #http://www.spatialreference.org/ref/epsg/2263/
geo_df = gpd.GeoDataFrame(df2, crs=crs, geometry=geometry) #points

CPU times: user 595 ms, sys: 56.7 ms, total: 652 ms
Wall time: 32.6 ms


In [5]:
geo_df.drop(columns=['Latitude','Longitude'],inplace=True)

In [6]:
%%time
coords = gpd.read_file('map_files/tl_2017_us_uac10.shp') #polygons
coords.drop(columns=['UACE10','NAME10','NAMELSAD10','LSAD10','MTFCC10','UATYP10','FUNCSTAT10','ALAND10',
                    'AWATER10','INTPTLAT10','INTPTLON10'],inplace=True)
#coords['RowIDs'] = [[] for _ in range(len(coords))]

CPU times: user 4.89 s, sys: 177 ms, total: 5.07 s
Wall time: 3.29 s


In [7]:
coords = gpd.read_file('map_files/tl_2017_us_uac10.shp',usecols='geometry') #polygons

In [8]:
geo_df['rural_Census'] = 1

In [9]:
def locate_it(pt):
    for id in coords.index:
        geo_df.loc[pt.within(coords.loc[id,'geometry']) == True] = 1
# Doesn't work

In [ ]:
%%time
geo_df['rural_Census'] = locate_it()

In [ ]:
def soc_iter(TEAM,home,away,ftr):
    df['Draws'] = 'No_Game'
    df.loc[((home == TEAM) & (ftr == 'D')) | ((away == TEAM) & (ftr == 'D')), 'Draws'] = 'Draw'
    df.loc[((home == TEAM) & (ftr != 'D')) | ((away == TEAM) & (ftr != 'D')), 'Draws'] = 'No_Draw'

df['Draws'] = soc_iter('Arsenal',df['HomeTeam'],df['AwayTeam'],df['FTR'])

In [ ]:
for i,v in coords.iterrows():
    if len(v['RowIDs']) != 0:
        print(coords.loc[i][['GEOID10','RowIDs']])

In [ ]:
s = coords[coords['GEOID10']=='44668']

In [ ]:
l = s['RowIDs'].to_list()

In [ ]:
len(l[0])